In [3]:
import yfinance as yf
import numpy as np
import pandas as pd
import datetime as dt
from dateutil.relativedelta import relativedelta
import alpaca_trade_api as tradeapi
import logging 

logging.basicConfig(level = logging.INFO)

# API credentials
api_key = ''
api_secret = ''
base_url = 'https://paper-api.alpaca.markets'  # Use 'https://api.alpaca.markets' for live trading

# Create an instance of the Alpaca API
api = tradeapi.REST(api_key, api_secret, base_url, api_version='v2')


def get_historical_data_from_api(symbol, start_date, end_date):
    # Retrieve historical price data using yfinance
    stock_data = yf.download(symbol, start=start_date, end=end_date, progress=False)
    
    return stock_data


def get_historical_data(symbol, account_balance, risk_percentage, entry_price, stop_loss_price):
    # Retrieve historical price data
    end_date = dt.date.today()
    start_date = end_date - relativedelta(years=3)
    
    start_date_str = start_date.strftime("%Y-%m-%d")
    end_date_str = end_date.strftime("%Y-%m-%d")
    
    # Replace this with your API call to retrieve historical price data
    historical_data = get_historical_data_from_api(symbol, start_date_str, end_date_str)
    
    # Preprocess the data
    historical_data = historical_data.dropna()  # Drop rows with missing values
    
    # Adjust data formats if necessary
    historical_data['Date'] = pd.to_datetime(historical_data.index)  # Convert index to datetime format
    
    # Scale values if needed (e.g., using Min-Max scaling)
    historical_data['Close'] = (historical_data['Close'] - historical_data['Close'].min()) / (historical_data['Close'].max() - historical_data['Close'].min())
    
    # Calculate additional derived features if desired
    historical_data['DailyReturns'] = historical_data['Close'].pct_change()  # Calculate daily returns
    
    # Calculate position size
    position_size = calculate_position_size(account_balance, risk_percentage, entry_price, stop_loss_price)
    
    return historical_data, position_size


def calculate_position_size(account_balance, risk_percentage, entry_price, stop_loss_price):
    # Calculate the maximum amount of risk based on the account balance and risk percentage
    max_risk_amount = account_balance * risk_percentage
    
    # Difference between the entry price and stop-loss price
    price_diff = entry_price - stop_loss_price
    
    # Calculate the number of shares to buy
    position_size = max_risk_amount / price_diff
    
    return position_size


# Call the get_historical_data function and assign the return values to variables
#symbol = 'AAPL'
#account_balance = 10000.0
#risk_percentage = 0.02
#entry_price = 150.0
#stop_loss_price = 140.0

#historical_data, position_size = get_historical_data(symbol, account_balance, risk_percentage, entry_price, stop_loss_price)

# Print the historical data
#print(historical_data)

# Print the position size
#print(position_size)

def open_trades():
    # Retrieve open trades from your trading account
    open_positions = api.list_positions()

    if open_positions:
        # Iterate over each open position and process the information
        for position in open_positions:
            symbol = position.symbol
            quantity = int(position.qty)
            side = position.side

            # Retrieve the latest trade price for the stock
            latest_trade = api.get_latest_trade(symbol)
            trade_price = float(latest_trade.p)

            # Perform any desired actions with the open trade data
            # For example, you can calculate unrealized profit/loss, check stop-loss levels, etc.
            unrealized_pl = (trade_price - float(position.avg_entry_price)) * quantity
            
             logging.info(f"Symbol: {symbol}")
            logging.info(f"Quantity: {quantity}")
            logging.info(f"Side: {side}")
            logging.info(f"Latest Trade Price: {trade_price}")
            logging.info(f"Unrealized P/L: {unrealized_pl}")
            logging.info("---")
    else:
        logging.info("No open trades.")


def place_entry_orders(open_positions, symbol, adjustment_quantity, side='buy', order_type='market', limit_price=None):
    existing_quantity = 0
    for position in open_positions:
        if position.symbol == symbol:
            existing_quantity = int(position.qty)
            break

    if existing_quantity != 0:
        if side == 'buy':
            logging.warning("Position already exists for the symbol. Close the existing position before opening a new one.")
            return None
        elif side == 'sell':
            if adjustment_quantity > existing_quantity:
                adjustment_quantity = existing_quantity
            order = api.submit_order(
                symbol=symbol,
                qty=adjustment_quantity,
                side=side,
                type=order_type,
                time_in_force='gtc',
                limit_price=limit_price
            )
            logging.info(f"Placed order to sell {adjustment_quantity} shares to close the existing position.")
            return order

    # Handle the scenario where no existing position needs to be closed
    if adjustment_quantity > 0:
        order = api.submit_order(
            symbol=symbol,
            qty=adjustment_quantity,
            side=side,
            type=order_type,
            time_in_force='gtc',
            limit_price=limit_price
        )
        logging.info(f"Placed order for {adjustment_quantity} shares.")
        return order

    return None
    
#testing place_entry_orders using historical data
#symbol = 'AAPL'
#account_balance = 10000.0
#risk_percentage = 0.02
#entry_price = 150.0
#stop_loss_price = 140.0

#historical_data, position_size = get_historical_data(symbol, account_balance, risk_percentage, entry_price, stop_loss_price)

# Print the historical data
#print(historical_data)

# Print the position size
#print(position_size)

#entry_order = place_entry_orders(symbol='AAPL', quantity=10, side='buy', order_type='limit', limit_price=155.0)

#######################################################

def place_exit_orders(symbol, quantity, side, order_type, limit_price = None):
    order = api.submit_order(
        symbol = symbol,
        qty = quantity,
        side = side, 
        type = order_type,
        time_in_force = 'gtc',
        limit_price = limit_price
    )
    return order


symbol = 'AAPL'
account_balance = 10000.0
risk_percentage = 0.02
entry_price = 150.0
stop_loss_price = 140.0

historical_data, position_size = get_historical_data(symbol, account_balance, risk_percentage, entry_price, stop_loss_price)

# Example usage of the place_entry_orders function
#entry_order = place_entry_orders(symbol='AAPL', quantity=10, side='buy', order_type='limit', limit_price=155.0)
#print(f"Entry Order ID: {entry_order.id}")

# Example usage of the place_exit_orders function
#exit_order = place_exit_orders(symbol='AAPL', quantity=10, side='sell', order_type='limit', limit_price=160.0)
#print(f"Exit Order ID: {exit_order.id}")

#portfolio managment
#retrive open positions, calculate unlreaised profits/Losses, Adjust position sizes, monitour account balance 
def portfolio_management(account_balance, risk_percentage, desired_positions):
    open_positions = api.list_positions()

    for pos in open_positions:
        symbol = pos.symbol
        quantity = int(pos.qty)
        side = pos.side

        # Retrieve the latest trade price for the stock
        latest_trade = api.get_latest_trade(symbol)
        trade_price = float(latest_trade.p)

        # Calculate unrealized profits/losses
        unrealized_pl = (trade_price - float(pos.avg_entry_price))

        # Print position info and unrealized profits/losses
        logging.info(f"Symbol: {symbol}")
        logging.info(f"Quantity: {quantity}")
        logging.info(f"Side: {side}")
        logging.info(f"Latest Trade Price: {trade_price}")
        logging.info(f"Unrealized P/L: {unrealized_pl}")
        logging.info("---")

        # Check for position adjustments
        if symbol in desired_positions:
            desired_quantity = desired_positions[symbol]
            if quantity != desired_quantity:
                # Calculate adjustment needed
                adjustment_quantity = desired_quantity - quantity

                if adjustment_quantity > 0:
                    place_entry_orders(open_positions, symbol, adjustment_quantity, 'buy', 'market')
                else:
                    place_entry_orders(open_positions, symbol, abs(adjustment_quantity), 'sell', 'market')

    # Monitor account balance and adjust position size
    max_risk_amount = account_balance * risk_percentage
    for symbol, desired_quantity in desired_positions.items():
        current_quantity = 0

IndentationError: unexpected indent (658272114.py, line 103)

In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
import datetime as dt
from dateutil.relativedelta import relativedelta
import alpaca_trade_api as tradeapi
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)

# API credentials
api_key = ''
api_secret = ''
base_url = 'https://paper-api.alpaca.markets'  # Use 'https://api.alpaca.markets' for live trading

# Create an instance of the Alpaca API
api = tradeapi.REST(api_key, api_secret, base_url, api_version='v2')


def get_historical_data_from_api(symbol, start_date, end_date):
    # Retrieve historical price data using yfinance
    stock_data = yf.download(symbol, start=start_date, end=end_date, progress=False)
    
    return stock_data


def get_historical_data(symbol, account_balance, risk_percentage, entry_price, stop_loss_price):
    # Retrieve historical price data
    end_date = dt.date.today()
    start_date = end_date - relativedelta(years=3)
    
    start_date_str = start_date.strftime("%Y-%m-%d")
    end_date_str = end_date.strftime("%Y-%m-%d")
    
    # Replace this with your API call to retrieve historical price data
    historical_data = get_historical_data_from_api(symbol, start_date_str, end_date_str)
    
    # Preprocess the data
    historical_data = historical_data.dropna()  # Drop rows with missing values
    
    # Adjust data formats if necessary
    historical_data['Date'] = pd.to_datetime(historical_data.index)  # Convert index to datetime format
    
    # Scale values if needed (e.g., using Min-Max scaling)
    historical_data['Close'] = (historical_data['Close'] - historical_data['Close'].min()) / (historical_data['Close'].max() - historical_data['Close'].min())
    
    # Calculate additional derived features if desired
    historical_data['DailyReturns'] = historical_data['Close'].pct_change()  # Calculate daily returns
    
    # Calculate position size
    position_size = calculate_position_size(account_balance, risk_percentage, entry_price, stop_loss_price)
    
    return historical_data, position_size


#Risk managment 
def check_max_drawdown(account_equity, max_drawdown_threshold):
    peak = np.maximum.accumalate(account_equity)
    drawdown = (account_equity - peak) / peak
    
    if np.min(drawdown) < max_drawdown_threshold:
        logging.info("Maximum drawdown exceeded. Adjusting risk exposure.")

def place_stop_loss_order(symbol, quantity, stop_price):
    order = api.submit_order(
        symbol=symbol,
        qty=quantity,
        side='sell',
        type='stop',
        time_in_force='gtc',
        stop_price=stop_price
    )
    return order

def calculate_position_size(account_balance, risk_percentage, entry_price, stop_loss_price):
    risk_percentage = 0.02  # Maximum risk percentage of 2% per trade

    # Calculate the maximum amount of risk based on the account balance and risk percentage
    max_risk_amount = account_balance * risk_percentage
    
    # Difference between the entry price and stop-loss price
    price_diff = entry_price - stop_loss_price
    
    # Calculate the number of shares to buy
    position_size = max_risk_amount / price_diff
    
    return position_size

def mean_reversion_strategy(data):
    #calculating mean and standard ditsribution
    mean = data['Close'].mean()
    std = data['Close'].std()
    
    #Define entry and exit thresholds
    entry_threshold = mean - std
    exit_threshold = mean + std
    
    #trading signals
    trading = []
    for i in range(1, len(data)):
        close_price = data['Close'].iloc[i]
        
        if close_price < entry_threshold:
            trading.append('buy')
        elif close_price > exit_threshold:
            trading.append('sell')
        else:
            trading.append('hold')
    return trading
    

def open_trades():
    # Retrieve open trades from your trading account
    open_positions = api.list_positions()

    if open_positions:
        # Iterate over each open position and process the information
        for position in open_positions:
            symbol = position.symbol
            quantity = int(position.qty)
            side = position.side

            # Retrieve the latest trade price for the stock
            latest_trade = api.get_latest_trade(symbol)
            trade_price = float(latest_trade.p)

            # Perform any desired actions with the open trade data
            # For example, you can calculate unrealized profit/loss, check stop-loss levels, etc.
            unrealized_pl = (trade_price - float(position.avg_entry_price)) * quantity

            logging.info(f"Symbol: {symbol}")
            logging.info(f"Quantity: {quantity}")
            logging.info(f"Side: {side}")
            logging.info(f"Latest Trade Price: {trade_price}")
            logging.info(f"Unrealized P/L: {unrealized_pl}")
            logging.info("---")
    else:
        logging.info("No open trades.")


def place_entry_orders(open_positions, symbol, adjustment_quantity, side='buy', order_type='market', limit_price=None):
    existing_quantity = 0
    for position in open_positions:
        if position.symbol == symbol:
            existing_quantity = int(position.qty)
            break

    if existing_quantity != 0:
        if side == 'buy':
            logging.warning("Position already exists for the symbol. Close the existing position before opening a new one.")
            return None
        elif side == 'sell':
            if adjustment_quantity > existing_quantity:
                adjustment_quantity = existing_quantity
            order = api.submit_order(
                symbol=symbol,
                qty=adjustment_quantity,
                side=side,
                type=order_type,
                time_in_force='gtc',
                limit_price=limit_price
            )
            logging.info(f"Placed order to sell {adjustment_quantity} shares to close the existing position.")
            return order

    # Handle the scenario where no existing position needs to be closed
    if adjustment_quantity > 0:
        order = api.submit_order(
            symbol=symbol,
            qty=adjustment_quantity,
            side=side,
            type=order_type,
            time_in_force='gtc',
            limit_price=limit_price
        )
        logging.info(f"Placed order for {adjustment_quantity} shares.")
        return order

    return None


def place_exit_orders(symbol, quantity, side, order_type, limit_price=None):
    order = api.submit_order(
        symbol=symbol,
        qty=quantity,
        side=side, 
        type=order_type,
        time_in_force='gtc',
        limit_price=limit_price
    )
    return order


def portfolio_management(account_balance, risk_percentage, desired_positions):
    open_positions = api.list_positions()

    for pos in open_positions:
        symbol = pos.symbol
        quantity = int(pos.qty)
        side = pos.side

        # Retrieve the latest trade price for the stock
        latest_trade = api.get_latest_trade(symbol)
        trade_price = float(latest_trade.p)

        # Calculate unrealized profits/losses
        unrealized_pl = (trade_price - float(pos.avg_entry_price))

        # Print position info and unrealized profits/losses
        logging.info(f"Symbol: {symbol}")
        logging.info(f"Quantity: {quantity}")
        logging.info(f"Side: {side}")
        logging.info(f"Latest Trade Price: {trade_price}")
        logging.info(f"Unrealized P/L: {unrealized_pl}")
        logging.info("---")

        # Check for position adjustments
        if symbol in desired_positions:
            desired_quantity = desired_positions[symbol]
            if quantity != desired_quantity:
                # Calculate adjustment needed
                adjustment_quantity = desired_quantity - quantity

                if adjustment_quantity > 0:
                    place_entry_orders(open_positions, symbol, adjustment_quantity, 'buy', 'market')
                else:
                    place_entry_orders(open_positions, symbol, abs(adjustment_quantity), 'sell', 'market')

    # Monitor account balance and adjust position size
    max_risk_amount = account_balance * risk_percentage
    for symbol, desired_quantity in desired_positions.items():
        current_quantity = 0
        

In [11]:
#backtesting 
def backtest_mean_reversion_strategy(symbol, start_date, end_date, account_balance, risk_percentage, entry_price, stop_loss_price):
    # Obtain historical data for backtesting
    historical_data, signals, position_size = get_historical_data(symbol, account_balance, risk_percentage, entry_price, stop_loss_price)
    
    # Initialize variables for tracking performance metrics
    total_trades = 0
    winning_trades = 0
    losing_trades = 0
    cumulative_returns = []

    # Iterate over the historical data and simulate trades
    for i in range(len(historical_data)):
        # Extract relevant data for the current time period
        close_price = historical_data['Close'].iloc[i]
        signal = signals[i]
        
        # Execute trades based on the signals
        if signal == 'buy':
            # Place buy order or execute desired action
            total_trades += 1
        elif signal == 'sell':
            # Place sell order or execute desired action
            total_trades += 1
        
        # Update performance metrics
        # Track profit/loss, winning/losing trades, etc.

    # Calculate performance metrics (e.g., cumulative returns, maximum drawdown)

    # Return performance metrics or save them for analysis